In [1]:
from haskellian import either as E
import pure_cv as vc
from moveread.core import CoreAPI
from moveread.export import player_samples, player_boxes
import chess_dataset as cds
import files_dataset as fds
import scoresheet_models as sm
import chess_notation as cn
import os
import fs

In [2]:
model_ids = (await sm.fetch_models()).unsafe()
models = {model_id: (await sm.fetch_model(model_id)).unsafe() for model_id in model_ids}

In [3]:
core = CoreAPI.at('.core/llobregat-b-1')
games = await core.games.items().map(E.unsafe).sync()

In [7]:
for i, (id, game) in enumerate(games):
  if not game.meta or not game.meta.tournament:
    continue
  tnmt = game.meta.tournament
  if not tnmt:
    print('NO TNMT:', id)
    continue

  if not game.meta.pgn:
    print('NO PGN:', id)
    continue

  path = 'data/' + f"{tnmt.tournId}/{tnmt.group}-{tnmt.round}-{tnmt.board}"
  os.makedirs(path, exist_ok=True)

  players = []
  for j, player in enumerate(game.players):
    boxes = await player_boxes(player, blobs=core.blobs, models=models)
    boxes = [s[0] for s in boxes]
    imgs = [vc.encode(box, '.jpg') for box in boxes]
    fs.create_tarfile([(f'box{i}.jpg', img) for i, img in enumerate(imgs)], path + f'/boxes{j}.tar')
    p = cds.Meta.Player(
      language=player.meta.language,
      styles=cn.Styles.model_validate(dict(player.meta.styles)),
      files_dataset={'boxes': fds.Archive(archive=f'boxes{j}.tar', num_files=len(boxes), format='tar')}
    )
    players.append(p)

  meta = cds.Meta(pgn=game.meta.pgn, players=players)
  with open(path + '/meta.json', 'w') as f:
    json = cds.MetaJson(chess_dataset=meta).model_dump_json(indent=2, exclude_none=True, exclude_defaults=True)
    f.write(json)